# Fine-tuning both vision and language parts

### There are problems with putting multimodal data in the model. Necessary to use special processor class object, but not important in our case

### Installing requirements

In [2]:
!pip install -q \
    torch \
    numpy \
    tqdm \
    torchvision \
    transformers \
    datasets \
    peft \
    accelerate \
    matplotlib \
    git+https://github.com/openai/CLIP.git \
    git+https://github.com/salaniz/pycocoevalcap
!pip install -q --upgrade bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0

### Necessary imports

In [3]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, get_scheduler, BitsAndBytesConfig
from datasets import load_dataset
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from accelerate import Accelerator
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import torchvision.transforms as T
from torchvision.transforms.functional import InterpolationMode
import bitsandbytes as bnb

2025-05-21 19:57:04.632412: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747857424.822280      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747857424.877128      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Defining variables

In [4]:
MODEL_NAME = "OpenGVLab/InternVL2_5-4B"
DATASET_NAME = "d0rj/LLaVA-OneVision-Data-ru"
DATASET_SUBDIR = "ureader_cap"
OUTPUT_DIR = "./finetuned_model"
BATCH_SIZE = 1
EPOCHS = 3
LR = 2e-4  # learning rate
MAX_LENGTH = 512
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
INPUT_SIZE = 448   #  for load_image()
MAX_NUM = 6       # max amount of tiles

### Uploading model and tokenizer

In [5]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True, use_fast=True)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# Model 4bit с bnb
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map='auto',
    quantization_config=quantization_config,
    trust_remote_code=True
)

# Preparing model for QLoRA
model = prepare_model_for_kbit_training(model)

tokenizer_config.json:   0%|          | 0.00/9.02k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.38M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/790 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.43G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

### LoRA configuration

In [6]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # for LLMs
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 3,686,400 || all params: 3,716,324,352 || trainable%: 0.0992


### Preparing the dataset

In [7]:
dataset = load_dataset(DATASET_NAME, DATASET_SUBDIR)

train_data = dataset["train"].select(range(1000))

# Learned normalization for images
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

README.md:   0%|          | 0.00/37.2k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/19 [00:00<?, ?it/s]

ureader_cap/train-00000-of-00019.parquet:   0%|          | 0.00/115M [00:00<?, ?B/s]

ureader_cap/train-00001-of-00019.parquet:   0%|          | 0.00/115M [00:00<?, ?B/s]

ureader_cap/train-00002-of-00019.parquet:   0%|          | 0.00/118M [00:00<?, ?B/s]

...

ureader_cap/train-00016-of-00019.parquet:   0%|          | 0.00/133M [00:00<?, ?B/s]

ureader_cap/train-00017-of-00019.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

ureader_cap/train-00018-of-00019.parquet:   0%|          | 0.00/109M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/91434 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/19 [00:00<?, ?it/s]

### Transformations pipeline

In [8]:
def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

### Finding the most appropriate aspect ratio for an image

In [9]:
def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

### Image preprocessing

In [10]:
def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

### Loading images

In [11]:
def load_image(image_file, input_size=448, max_num=12):
    image = image_file.convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

### General preprocessing

In [12]:
def preprocess_function(examples):
    input_ids_list = []
    attention_mask_list = []
    pixel_values_list = []
    labels_list = []

    for convs, image in zip(examples["conversations"], examples["image"]):
        full_text = ""
        label_mask = []

        for turn in convs:
            if turn["from"] == "human":
                text = turn["value"]
                full_text += text + "\n"
                label_mask += [0] * len(tokenizer.tokenize(text + "\n"))
            elif turn["from"] == "gpt":
                text = turn["value"]
                full_text += text + "\n"
                label_mask += [1] * len(tokenizer.tokenize(text + "\n"))

        tokenized = tokenizer(
            full_text,
            max_length=MAX_LENGTH,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        input_ids = tokenized["input_ids"].squeeze(0)        # tensor
        attention_mask = tokenized["attention_mask"].squeeze(0)  # tensor

        label_mask = label_mask[:MAX_LENGTH]
        label_mask += [0] * (MAX_LENGTH - len(label_mask))

        labels_tensor = input_ids.clone()  # cloning tensor
        for i, mask in enumerate(label_mask):
            if mask == 0:
                labels_tensor[i] = -100

        pixel_values = load_image(image, input_size=INPUT_SIZE, max_num=MAX_NUM)
        pixel_values = [p.tolist() for p in pixel_values]

        input_ids_list.append(input_ids)
        attention_mask_list.append(attention_mask)
        pixel_values_list.append(pixel_values)
        labels_list.append(labels_tensor)

    batch_input_ids = torch.stack(input_ids_list)
    batch_attention_mask = torch.stack(attention_mask_list)
    # pixel_values — list
    batch_labels = torch.stack(labels_list)

    return {
        "input_ids": batch_input_ids,
        "attention_mask": batch_attention_mask,
        "pixel_values": pixel_values_list,
        "labels": batch_labels,
    }

In [13]:
processed_dataset = train_data.map(
    preprocess_function,
    batched=True,
    batch_size=16,
    remove_columns=train_data.column_names,
    load_from_cache_file=False
)

# Splitting on training/validation
split = processed_dataset.train_test_split(test_size=0.1)
train_dataset = split["train"]
val_dataset = split["test"]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

### Collating

In [14]:
def collate_fn(batch):
    input_ids_list = []
    attention_mask_list = []
    labels_list = []
    pixel_values_list = []

    for sample in batch:
        input_ids_list.append(torch.tensor(sample['input_ids'], dtype=torch.long))
        attention_mask_list.append(torch.tensor(sample['attention_mask'], dtype=torch.long))
        labels_list.append(torch.tensor(sample['labels'], dtype=torch.long))

        pv = torch.stack([torch.tensor(p) for p in sample['pixel_values']])
        pixel_values_list.append(pv)

    return {
        'input_ids': torch.nn.utils.rnn.pad_sequence(input_ids_list, batch_first=True, padding_value=tokenizer.pad_token_id),
        'attention_mask': torch.nn.utils.rnn.pad_sequence(attention_mask_list, batch_first=True, padding_value=0),
        'labels': torch.nn.utils.rnn.pad_sequence(labels_list, batch_first=True, padding_value=-100),
        'pixel_values': torch.stack(pixel_values_list)
    }

### Batching

In [15]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

### Defining metrics

In [16]:
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.spice.spice import Spice
import clip

In [17]:
cider_scorer = Cider()
spice_scorer = Spice()
clip_model, clip_preprocess = clip.load("ViT-B/32", device=DEVICE)

Progress: 384.5M / 384.5M (100.0%)
Extracting stanford-corenlp-3.6.0 ...
Done.


100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 126MiB/s]


### Computing CLIPScore

In [18]:
def compute_clip_score(cand_sentences, ref_sentences, batch_size=16):
    scores = []
    for i in range(0, len(cand_sentences), batch_size):
        batch_cand = cand_sentences[i:i+batch_size]
        batch_ref = ref_sentences[i:i+batch_size]
        cand_inputs = clip.tokenize(batch_cand).to(DEVICE)
        ref_inputs = clip.tokenize(batch_ref).to(DEVICE)
        cand_feats = clip_model.encode_text(cand_inputs)
        ref_feats = clip_model.encode_text(ref_inputs)
        cand_feats = cand_feats / cand_feats.norm(dim=-1, keepdim=True)
        ref_feats = ref_feats / ref_feats.norm(dim=-1, keepdim=True)
        scores.extend((cand_feats * ref_feats).sum(dim=-1).tolist())
    return np.mean(scores)

### Optimizer, scheduler, accelerator

In [19]:
# Optimizer and scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
num_training_steps = EPOCHS * len(train_loader)
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Accelerator
accelerator = Accelerator()
model, optimizer, train_loader, val_loader, lr_scheduler = accelerator.prepare(
    model, optimizer, train_loader, val_loader, lr_scheduler
)

### Final preparations

### Generating text for validation

In [20]:
def generate_text(input_ids, attention_mask):
    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=MAX_LENGTH,
        num_beams=1,
        early_stopping=True,
    )
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

### Training loop

Here is the problem

In [21]:
train_losses = []
val_losses = []
spice_scores = []
cider_scores = []
clip_scores = []

In [22]:
for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}/{EPOCHS}")
    model.train()
    total_train_loss = 0
    for batch in tqdm(train_loader):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Валидация
    model.eval()
    total_val_loss = 0
    preds = []
    refs = []
    with torch.no_grad():
        for batch in tqdm(val_loader):
            outputs = model(**batch)
            loss = outputs.loss
            total_val_loss += loss.item()

            # Генерируем предсказания для метрик
            generated_texts = generate_text(batch["input_ids"], batch["attention_mask"])
            refs.extend(tokenizer.batch_decode(batch["labels"], skip_special_tokens=True))
            preds.extend(generated_texts)

    avg_val_loss = total_val_loss / len(val_loader)
    val_losses.append(avg_val_loss)

    # Подсчет SPICE и CIDEr (pycocoevalcap работает со словарями)
    # Формат: [{'image_id': i, 'caption': 'text'}, ...]
    res = [{"image_id": i, "caption": p} for i, p in enumerate(preds)]
    gts = {i: [refs[i]] for i in range(len(refs))}

    cider_score, _ = cider_scorer.compute_score(gts, res)
    spice_score, _ = spice_scorer.compute_score(gts, res)

    cider_scores.append(cider_score)
    spice_scores.append(spice_score)

    # CLIPScore
    clip_score = compute_clip_score(preds, refs)
    clip_scores.append(clip_score)

    print(f"Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | SPICE: {spice_score:.4f} | CIDEr: {cider_score:.4f} | CLIPScore: {clip_score:.4f}")


Epoch 1/3


  0%|          | 0/900 [00:04<?, ?it/s]


TypeError: InternVLChatModel.forward() got an unexpected keyword argument 'inputs_embeds'

### Saving model

In [ ]:
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"Model is saved: {OUTPUT_DIR}")

### Graphical visualization

In [ ]:
epochs = np.arange(1, EPOCHS+1)

plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(epochs, train_losses, label="Train Loss")
plt.plot(epochs, val_losses, label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.title("Loss")

plt.subplot(2, 2, 2)
plt.plot(epochs, spice_scores, label="SPICE", color="orange")
plt.xlabel("Epoch")
plt.ylabel("Score")
plt.legend()
plt.title("SPICE")

plt.subplot(2, 2, 3)
plt.plot(epochs, cider_scores, label="CIDEr", color="green")
plt.xlabel("Epoch")
plt.ylabel("Score")
plt.legend()
plt.title("CIDEr")

plt.subplot(2, 2, 4)
plt.plot(epochs, clip_scores, label="CLIPScore", color="red")
plt.xlabel("Epoch")
plt.ylabel("Score")
plt.legend()
plt.title("CLIPScore")

plt.tight_layout()
plt.show()